In [2]:
class Sequence:
    """
    Represents a potentially cached and infinite sequence of numbers that can be generated by a function, by batch computation, and by a recursive formula.
    """
    
    
    def __init__(self, func, offset = 0, cache = True):
        
        self.offset = offset
        
        if cache:
            
            cache_data = {}
            
            def atn(n):
                
                if not n in cache_data:
                    cache_data[n] = func(n)
                
                return cache_data[n]
                
                
            
            self.func = atn
            
        else:
            
            self.func = func
            
        
    
    def __call__(self, n):
        return self.func(n)
    
    def to_list(self, *args):
        """
        Args are simply passed to IntegerRange and then used as a sampling of the sequence.
        """
        return self.to_list_with_sampling(IntegerRange(*args))
    
    def to_list_with_sampling(self, sampling):
        return [self(s) for s in sampling]
    
    def test_multiplicative(self, terms = 50000, prec = 0):
        
        for n in IntegerRange(1, terms + 1):
            if n % 100 == 0:
                print("\r" + str(round(100 * (n) / (terms), 2)) + "% Done     ", end="")
            
            actual = self(n)
            theoretical = product(self(p^e) for p, e in n.factor())
            
            if abs(actual - theoretical) > prec:
                return False
            
        print("\r100.0% Done")
        
        return True
    
    def test_completely_multiplicative(self, terms = 50000, prec = 0):
        
        for n in IntegerRange(1, terms + 1):
            if n % 100 == 0:
                clear_output(wait=True)
                print("\r" + str(round(100 * (n) / (terms - 1), 2)) + "% Done     ", end="")
            
            actual = self(n)
            theoretical = product(self(p)^e for p, e in n.factor())
            
            if abs(actual - theoretical) > prec:
                return False
            
        print("\r100.0% Done")
        
        return True
    
    
    def find_pade(self, maxupstairs, maxdownstairs, verifyterms = None, prec = 10^(-15), ring = CC):
        if verifyterms == None:
            verifyterms = maxupstairs + maxdownstairs + 1
        
        F.<t> = PowerSeriesRing(ring)
        
        gf = O(t^(verifyterms + 1)) + sum(t^n * ring(self(n - self.offset)) for n in IntegerRange(verifyterms + 1))
        
        gfpade = gf.pade(maxdownstairs, maxupstairs)
        
        gfpadeexp = gfpade(t + O(t^verifyterms + 1))
        
        diffcoeffs = (gf - gfpadeexp).padded_list()
        
        if all(abs(c) < prec for c in diffcoeffs):
            return gfpade
        else:
            return None
        

    def gtate_twist(self, s):
        return create_sequence(lambda n: self(n) * s.parent()(n) ^ (s), offset = self.offset)
        
    def dirichlet_convolution(self, other):
        return create_sequence(lambda n: sum(self(d) * other(ZZ(n/d)) for d in divisors(n)), offset = max(self.offset, other.offset))
    
    def dirichlet_inverse(self):
        def an_comp(rec, n):
            if n == 1:
                return 1/self(1)
            s = 0
            for d in divisors(n):
                if d == n:
                    continue
                s += self(ZZ(n/d)) * rec(d)
            return -s/self(1)
        
        return create_sequence_recursive(an_comp, offset = self.offset)
        
    
    def pointwise_product(self, other):
        return create_sequence(lambda n: self(n) * other(n), offset = max(self.offset, other.offset))
        
    def pointwise_sum(self, other):
        return create_sequence(lambda n: self(n) + other(n), offset = max(self.offset, other.offset))
        
    def scale(self, factor):
        return create_sequence(lambda n: factor * self(n), offset = self.offset)
        
    def power(self, expo):
        return create_sequence(lambda n: self(n) ^ expo, offset = self.offset)
    
    def gbell_derivative(self):
        return self.pointwise_product(create_sequence(lambda n: number_of_divisors(n), offset = 1)).dirichlet_convolution(self.dirichlet_inverse())
    
    def gbell_antiderivative(self):
        if self(1) != 1 or self.offset != 1:
            raise ValueError("Bell antiderivative of sequences not starting with 1 and offset 1 has not been properly implemented!")

        def an_comp(rec, n):
            if n == 1:
                return 1

            tau = number_of_divisors(n)

            s = 0
            for d in divisors(n):
                if d == n:
                    continue
                s += self(ZZ(n/d)) * rec(d)

            return 1/(tau - 1) * s
        
        return create_sequence_recursive(an_comp, offset = self.offset)
        
    
    def oplus(self, other):
        return self.dirichlet_convolution(other)
    
    def oneg(self):
        return self.dirichlet_inverse()
    
    def ominus(self, other):
        return self.oplus(other.oneg())
    
    def otimes(self, other):
        return (self.gbell_derivative().boxtimes(other.gbell_derivative())).gbell_antiderivative()
    
    def boxplus(self, other):
        return (self.gbell_antiderivative().oplus(other.gbell_antiderivative())).gbell_derivative()
    
    def boxneg(self):
        return (self.gbell_antiderivative().oneg()).gbell_derivative()
    
    def boxminus(self, other):
        return (self.gbell_antiderivative().ominus(other.gbell_antiderivative())).gbell_derivative()
    
    def boxtimes(self, other):
        return self.pointwise_product(other)
    
    def lbell_derivative(self):
        return create_sequence_recursive(lambda rec, n: 1 if n == 0 else n * self(n) - sum(self(j) * rec(n - j) for j in IntegerRange(1, n)))
    

    def lbell_antiderivative(self):
        return create_sequence_recursive(lambda rec, n: 1 if n == 0 else (self(n) + sum(rec(j) * self(n - j) for j in IntegerRange(1, n))) / n)
    
    def find_in_terms_of(self, others_with_name_cost, maxcost = 5, prec = 10^-15, verifyterms = 30, unary_ops_with_name_cost = None, binary_ops_with_name_cost = None):
        if unary_ops_with_name_cost == None:
            unary_ops_with_name_cost = [
                (1, "gbell_derivative", lambda seq: seq.gbell_derivative()),
                (1, "gbell_antiderivative", lambda seq: seq.gbell_antiderivative()),
                #(2, "lbell_derivative", lambda seq: seq.lbell_derivative()),
                #(2, "lbell_antiderivative", lambda seq: seq.lbell_antiderivative()),
                (1, "oneg", lambda seq: seq.oneg()),
                (1, "boxneg", lambda seq: seq.boxneg()),
                (2, "power2", lambda seq: seq.power(2)),
                (3, "power3", lambda seq: seq.power(3)),
                (1, "tate1", lambda seq: seq.gtate_twist(1)),
                (2, "tate2", lambda seq: seq.gtate_twist(2)),
            ]
            
        if binary_ops_with_name_cost == None:
            binary_ops_with_name_cost = [
                (1, "oplus", lambda seq1, seq2: seq1.oplus(seq2)),
                (1, "ominus", lambda seq1, seq2: seq1.ominus(seq2)),
                (2, "otimes", lambda seq1, seq2: seq1.otimes(seq2)),
                (2, "boxplus", lambda seq1, seq2: seq1.boxplus(seq2)),
                (2, "boxminus", lambda seq1, seq2: seq1.boxminus(seq2)),
                (1, "boxtimes", lambda seq1, seq2: seq1.boxtimes(seq2)),
            ]
        
        costpool = {n: [] for n in IntegerRange(1, maxcost + 1)}
        
        def add_to_pool(seq, name, cost):
            costpool[cost].append((seq, name))
            
            
        for seq, name, cost in others_with_name_cost:
            add_to_pool(seq, name, cost)
            
            if self.is_equal(seq, verifyterms = verifyterms, prec = prec):
                yield (seq, name, cost)
            
        min_cost = 1
        
        for current_cost in IntegerRange(1, maxcost + 1):
            for unary_op in unary_ops_with_name_cost:
                uo_cost, uo_name, uo_op = unary_op
                
                if uo_cost + min_cost <= current_cost:
                    seq_cost = current_cost - uo_cost
                    seqs = costpool[seq_cost]
                    
                    for seq_data in seqs:
                        seq, seq_name = seq_data
                        
                        nseq = uo_op(seq)
                        nseq_name = uo_name + "(" + seq_name + ")"
                        
                        add_to_pool(nseq, nseq_name, current_cost)
                        
                        
                        if self.is_equal(nseq, verifyterms = verifyterms, prec = prec):
                            yield (nseq, nseq_name, current_cost)
            
            for binary_op in binary_ops_with_name_cost:
                bo_cost, bo_name, bo_op = binary_op
                
                if bo_cost + 2 * min_cost <= current_cost:
                    for seq1_cost in IntegerRange(min_cost, current_cost - bo_cost - min_cost + 1):
                        seq2_cost = current_cost - bo_cost - seq1_cost
                        
                        seq1s = costpool[seq1_cost]
                        seq2s = costpool[seq2_cost]
                        
                        for seq1_data in seq1s:
                            seq1, seq1_name = seq1_data
                            
                            for seq2_data in seq2s:
                                seq2, seq2_name = seq2_data
                                
                                nseq = bo_op(seq1, seq2)
                                nseq_name = bo_name + "(" + seq1_name + ", " + seq2_name + ")"
                                
                                add_to_pool(nseq, nseq_name, current_cost)
                                
                                
                                if self.is_equal(nseq, verifyterms = verifyterms, prec = prec):
                                    yield (nseq, nseq_name, current_cost)
        
            print("Completed cost: ", current_cost)
                    
                
        
        
    
    def is_equal(self, other, verifyterms = 10000, prec = 10^-15):
        if self.offset != other.offset:
            return False
        
        return all(abs(self(n) - other(n)) < prec for n in IntegerRange(self.offset, self.offset + verifyterms))
    
    
    def __repr__(self):
        return "Sequence " + ", ".join([str(self(n)) for n in IntegerRange(self.offset, 6 + self.offset)]) + " (offset " + str(self.offset) + ")"
        
        
    

In [3]:
def create_sequence(func, offset = 0):
    return Sequence(func, offset = offset)

def create_sequence_batched(func_list_first_n, offset = 0, batch_next_n = None, default_batch_size = 10000):
    
    batch_cache = []
    
    if batch_next_n == None:
        
        batch_next_n = lambda n: default_batch_size * (1 + ((n - offset + 1) // default_batch_size))
        
    def at(n):
        
        nonlocal batch_cache
        
        n1 = n - offset
        
        if n1 < 0:
            return None
        
        if n1 < len(batch_cache):
            return batch_cache[n1]
        else:
            
            batch_cache = func_list_first_n(batch_next_n(n1))
            return batch_cache[n1]
    
    return Sequence(at, offset = offset, cache = True)

def create_sequence_recursive(recursive_func, offset = 0):
    """
    recursive_func is a function (N -> C), N -> C, where it is essentially passed itself.
    """
    
    recursive_cache = {}
    
    
    
    def at(n):
        
        if not n in recursive_cache:
            recursive_cache[n] = recursive_func(at, n)
        
        return recursive_cache[n]
        
    return Sequence(at, offset = offset, cache = False)

def create_sequence_from_list(ans, offset = 0):
    
    def at(n):
        return ans[n - offset]

        
    return Sequence(at, offset = offset, cache = False)

In [9]:
def compare_sequences(seq1, seq2, N = 10):
    if seq1.offset != seq2.offset:
        raise ValueError("Sequences must start at the same number to be compared!")
    
    o = seq1.offset
    return table([seq1.to_list(o, N + o), seq2.to_list(o, N + o)])